--Download library--

In [1]:
!pip install corextopic

--import dependencies--

In [75]:
import numpy as np
import pandas as pd

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS
import en_core_web_lg

from pprint import pprint

--read in data--

In [167]:
doc_anx = pd.read_csv(r'2020_March_r_Anxiety.csv')
doc_anx = doc_anx[doc_anx.Body != '[removed]']

text_doc_anx = doc_anx['Body'].tolist()

In [168]:
doc_list_anx = []
# Iterates through each article in the corpus.
for doc in text_doc_anx:
    # Passes that article through the pipeline and adds to a new list.
    pr = nlp(str(doc))
    doc_list_anx.append(pr)

In [169]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
empt = [];
for doc in doc_list_anx:
    a1 = TreebankWordDetokenizer().detokenize(doc)
    empt.append(a1)
    
df_doc_anx = pd.DataFrame(empt,columns = ['Body'])
#df_doc is a dataframe that has cleaned posts. 
#all the 'removed' posts are gone and all the stopwords in the individual posts are gone! 

#both doc_list and df_doc are important (at least I think so :D)


df_doc_anx

,Body
0,thing anxious focus mean study like maybe lazy...
1,know midnight know homework know school know X...
2,med euphoric feeling consider narcotic
3,heart feel sense yesterday continually worried...
4,fear swallow small bone eat chicken sure bite ...
...,...
7149,want sell ps4 use anymore want money hard pers...
7150,yesterday(this morning sleep midnight 12 wake ...
7151,bath new house cause flood basement live boyfr...
7152,mean find self cross leg


In [180]:
df_doc_anx['label']=1
df_doc_anx

,Body,label
0,thing anxious focus mean study like maybe lazy...,1
1,know midnight know homework know school know X...,1
2,med euphoric feeling consider narcotic,1
3,heart feel sense yesterday continually worried...,1
4,fear swallow small bone eat chicken sure bite ...,1
...,...,...
7149,want sell ps4 use anymore want money hard pers...,1
7150,yesterday(this morning sleep midnight 12 wake ...,1
7151,bath new house cause flood basement live boyfr...,1
7152,mean find self cross leg,1


In [172]:
doc = pd.read_csv(r'Datasets/2020_March_r_Depression.csv')
doc = doc[doc.Body != '[removed]']
doc = doc.sample(n=7154) 

In [173]:
#create a list of documents (list of lists) 
text_doc = doc['Body'].tolist()


In [107]:
nlp= spacy.load("en")

# List of stop words to equalize data
stop_list = ["Depression","depression",  "anxiety", "Anxiety"]

# Updates spaCy's default stop words list with my additional words. 
nlp.Defaults.stop_words.update(stop_list)

# Iterates over the words in the stop words list and resets the "is_stop" flag.
for word in STOP_WORDS:
    lexeme = nlp.vocab[word]
    lexeme.is_stop = True

In [108]:
def lemmatizer(doc):
    # This takes in a doc of tokens from the NER and lemmatizes them. 
    # Pronouns (like "I" and "you" get lemmatized to '-PRON-', so I'm removing those.
    doc = [token.lemma_ for token in doc if token.lemma_ != '-PRON-']
    doc = u' '.join(doc)
    return nlp.make_doc(doc)
    
def remove_stopwords(doc):
    # This will remove stopwords and punctuation.
    # Use token.text to return strings, which we'll need for Gensim.
    doc = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
    return doc

# The add_pipe function appends our functions to the default pipeline.
nlp.add_pipe(lemmatizer,name='lemmatizer',after='ner')
nlp.add_pipe(remove_stopwords, name="stopwords", last=True)

In [174]:
doc_list = []
# Iterates through each article in the corpus.
for doc in text_doc:
    # Passes that article through the pipeline and adds to a new list.
    pr = nlp(str(doc))
    doc_list.append(pr)

In [175]:
#however doc_list gives us each post as a list, with individual words being elements 
from nltk.tokenize.treebank import TreebankWordDetokenizer
empt = [];
for doc in doc_list:
    a1 = TreebankWordDetokenizer().detokenize(doc)
    empt.append(a1)
    
df_doc = pd.DataFrame(empt,columns = ['Body'])
#df_doc is a dataframe that has cleaned posts. 
#all the 'removed' posts are gone and all the stopwords in the individual posts are gone! 

#both doc_list and df_doc are important (at least I think so :D)


df_doc

,Body
0,thank come ted talk
1,cry thing heavy feel like cry company rude lea...
2,honestly lazy motivated life better like want ...
3,day know reason live exist feel like fall endl...
4,I‘ve depress 14 today 22nd birthday finally ma...
...,...
7149,feel like awful person rant know million peopl...
7150,fuck literally die Jesus christ feel brain spl...
7151,hate right stupid thing hate add Dyslexia Anzi...
7152,extremely happy able kill 15 min thought patte...


In [182]:
df_doc['label'] = 0
df_doc

,Body,label
0,thank come ted talk,0
1,cry thing heavy feel like cry company rude lea...,0
2,honestly lazy motivated life better like want ...,0
3,day know reason live exist feel like fall endl...,0
4,I‘ve depress 14 today 22nd birthday finally ma...,0
...,...,...
7149,feel like awful person rant know million peopl...,0
7150,fuck literally die Jesus christ feel brain spl...,0
7151,hate right stupid thing hate add Dyslexia Anzi...,0
7152,extremely happy able kill 15 min thought patte...,0


In [194]:
df_col_merged =pd.concat([df_doc, df_doc_anx], axis=0).reset_index(drop=True)
df_col_merged

,Body,label
0,thank come ted talk,0
1,cry thing heavy feel like cry company rude lea...,0
2,honestly lazy motivated life better like want ...,0
3,day know reason live exist feel like fall endl...,0
4,I‘ve depress 14 today 22nd birthday finally ma...,0
...,...,...
14303,want sell ps4 use anymore want money hard pers...,1
14304,yesterday(this morning sleep midnight 12 wake ...,1
14305,bath new house cause flood basement live boyfr...,1
14306,mean find self cross leg,1


In [195]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
    max_df=.5,
    min_df=10,
    max_features=None,
    ngram_range=(1, 1),
    norm=None,
    binary=True,
    use_idf=False,
    sublinear_tf=False
)

vectorizer = vectorizer.fit(df_col_merged.Body)
tfidf = vectorizer.transform(df_col_merged.Body)
vocab = vectorizer.get_feature_names()
print(len(vocab))

4547


In [196]:
tfidf

<14308x4547 sparse matrix of type '<class 'numpy.float64'>'
	with 576285 stored elements in Compressed Sparse Row format>

In [197]:
from corextopic import corextopic as ct
anchors = []
model = ct.Corex(n_hidden=6, seed=42)
model = model.fit(
    tfidf,
    words=vocab
)


In [201]:
X = tfidf;
y = df_col_merged.label.tolist()

import numpy as np
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0).fit(X_train, y_train)


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [202]:
clf.score(X_test,y_test)

0.7959468902865129

In [198]:

for i, topic_ngrams in enumerate(model.get_topics(n_words=15)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

Topic #1: life, friend, people, thing, good, think, talk, way, love, person, care, happy, look, understand, hate
Topic #2: start, day, panic, attack, week, month, doctor, ago, night, heart, experience, sleep, hour, chest, symptom
Topic #3: try, find, help, bad, point, hard, read, thought, mind, mental, problem, post, feeling, struggle, write
Topic #4: year, know, want, school, tell, end, relationship, old, break, away, college, anymore, ask, kid, lose
Topic #5: time, work, come, need, lot, long, health, issue, new, able, use, right, situation, sure, high
Topic #6: job, home, leave, live, family, pay, money, parent, house, stay, place, mom, let, room, walk


In [162]:
topic_df = pd.DataFrame(
    model.transform(tfidf), 
    columns=["topic_{}".format(i+1) for i in range(6)]
).astype(float)
topic_df.iloc[0:10]

,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6
0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0
5,1.0,1.0,0.0,0.0,0.0,1.0
6,1.0,1.0,1.0,1.0,1.0,1.0
7,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0
9,1.0,0.0,0.0,0.0,0.0,0.0


In [154]:
# Anchors designed to nudge the model towards measuring specific genres
anchors = [
    ["family"],
    ["die"],
    [""],
    [""],
    [""],
    [""],

]
anchors = [
    [a for a in topic if a in vocab]
    for topic in anchors
]

model = ct.Corex(n_hidden=8, seed=42)
model = model.fit(
    tfidf,
    words=vocab,
    anchors=anchors, # Pass the anchors in here
    anchor_strength=3 # Tell the model how much it should rely on the anchors
)

In [95]:
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

Topic #1: relationship, year, friend, start, love, ago, month, girl, meet, break
Topic #2: anymore, die, kill, want die, know anymore, want kill, anymore feel, anymore want, want anymore, fucking
Topic #3: family, mom, dad, old, parent, friend family, year old, mother, brother, sister
Topic #4: work, money, job, pay, work hard, bill, rent, school work, work feel, car
Topic #5: play, enjoy, game, watch, video, video game, spend, play video, sit, movie
Topic #6: school, motivation, high, college, lose, home, high school, class, able, anxiety
Topic #7: time, thing, try, life, good, day, come, help, find, long
Topic #8: know, think, people, want, tell, talk, feel like, bad, way, person


In [104]:
topic_df = pd.DataFrame(
    model.transform(tfidf), 
    columns=["topic_{}".format(i+1) for i in range(6)]
).astype(float)
topic_df

,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...
17546,0.0,0.0,0.0,0.0,0.0,0.0
17547,1.0,0.0,0.0,0.0,0.0,1.0
17548,0.0,1.0,1.0,1.0,1.0,0.0
17549,0.0,0.0,0.0,0.0,0.0,0.0


--LDA UNSUPERVISED--

In [74]:
# Creates, which is a mapping of word IDs to words.
words = corpora.Dictionary(doc_list)

# Turns each document into a bag of words.
corpus = [words.doc2bow(doc) for doc in doc_list]

In [33]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=words,
                                           num_topics=6, 
                                           random_state=2,
                                           update_every=1,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [13]:
#print the keyword in the 10 topics
pprint(lda_model.print_topics(num_words=100))

[(0,
  '0.107*"\n'
  '\n'
  ' " + 0.013*"life" + 0.009*"love" + 0.008*"look" + 0.007*"play" + '
  '0.007*"game" + 0.005*"world" + 0.005*"thing" + 0.005*"watch" + '
  '0.005*"amp;#x200b" + 0.004*"video" + 0.004*"man" + 0.004*"good" + '
  '0.004*"learn" + 0.004*"way" + 0.004*"people" + 0.004*"music" + 0.004*"time" '
  '+ 0.004*"find" + 0.003*"remember" + 0.003*"year" + 0.003*"grow" + '
  '0.003*"old" + 0.003*"dream" + 0.003*"read" + 0.003*"enjoy" + 0.003*"day" + '
  '0.003*"face" + 0.003*"art" + 0.003*"use" + 0.003*"child" + 0.003*"mind" + '
  '0.003*"self" + 0.003*"book" + 0.003*"come" + 0.003*"kid" + 0.003*"body" + '
  '0.002*"awful" + 0.002*"believe" + 0.002*"human" + 0.002*"movie" + '
  '0.002*"eye" + 0.002*"moment" + 0.002*"little" + 0.002*"woman" + '
  '0.002*"hair" + 0.002*"spend" + 0.002*"lack" + 0.002*"tv" + 0.002*"create" + '
  '0.002*"change" + 0.002*"realize" + 0.002*"beautiful" + 0.002*"live" + '
  '0.002*"memory" + 0.002*"draw" + 0.002*"step" + 0.002*"big" + 0.002*"try" + '

In [12]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=doc_list):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=doc_list)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(100)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,3.0,0.3478,"friend, tell, talk, know, \n\n , time, like, y...","[Uk, cahms, service, shit, sure, schizophrenia..."
1,1,5.0,0.4429,"\n\n , work, year, job, depression, time, week...","[anybody, luck, holistic, treatment, gut, heal..."
2,2,2.0,0.7067,"feel, like, want, know, life, think, thing, \n...","[leave, bed, 4:20pm, work, 5:00pm, feel, like,..."
3,3,2.0,0.6437,"feel, like, want, know, life, think, thing, \n...","[want, help, people, care, feel, right, like, ..."
4,4,2.0,0.9316,"feel, like, want, know, life, think, thing, \n...","[obviously, want, unhappy, want, depressed, mi..."
...,...,...,...,...,...
95,95,2.0,0.3870,"feel, like, want, know, life, think, thing, \n...",[remove]
96,96,2.0,0.4786,"feel, like, want, know, life, think, thing, \n...",[]
97,97,3.0,0.3257,"friend, tell, talk, know, \n\n , time, like, y...","[snapchat, caption, lol, https://i.imgur.com/s..."
98,98,3.0,0.3856,"friend, tell, talk, know, \n\n , time, like, y...","[grandma, 300, $, acoustic, guitar, plug, amp,..."


In [14]:
df_dominant_topic.head(25)
#df_dominant_topic.to_csv(r'depression_2020_datest.csv')

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,3.0,0.3478,"friend, tell, talk, know, \n\n , time, like, y...","[Uk, cahms, service, shit, sure, schizophrenia..."
1,1,5.0,0.4429,"\n\n , work, year, job, depression, time, week...","[anybody, luck, holistic, treatment, gut, heal..."
2,2,2.0,0.7067,"feel, like, want, know, life, think, thing, \n...","[leave, bed, 4:20pm, work, 5:00pm, feel, like,..."
3,3,2.0,0.6437,"feel, like, want, know, life, think, thing, \n...","[want, help, people, care, feel, right, like, ..."
4,4,2.0,0.9316,"feel, like, want, know, life, think, thing, \n...","[obviously, want, unhappy, want, depressed, mi..."
5,5,2.0,0.4599,"feel, like, want, know, life, think, thing, \n...","[watch, countless, reddit, video, topic, creat..."
6,6,2.0,0.3061,"feel, like, want, know, life, think, thing, \n...","[know, write, feel, like, place, thing, moment..."
7,7,2.0,0.8425,"feel, like, want, know, life, think, thing, \n...","[society, fucking, want, dead, wish, honestly,..."
8,8,2.0,0.4307,"feel, like, want, know, life, think, thing, \n...","[\n , 26, yo, \n\n , ready, honestly, end, hel..."
9,9,2.0,0.7250,"feel, like, want, know, life, think, thing, \n...","[Indica, edible, Wednesday, feel, amazing, usu..."


In [15]:
from collections import Counter
Counter(df_dominant_topic.Dominant_Topic)

Counter({3.0: 3007, 5.0: 2454, 2.0: 13603, 0.0: 301, 1.0: 97, 4.0: 18})

In [16]:
newest_doc = newest_doc[newest_doc.Body != '[removed]'].dropna()

In [17]:
newest_doc.Body

0        Uk CAHMS service is shit. I’m sure I have schi...
1        Has anybody had any luck with holistic treatme...
2        Didn’t leave bed until 4:20pm so I could go to...
3        I want help, I can't say to the people that ca...
4        Obviously I don't want to be unhappy. I don't ...
                               ...                        
19474    I finally open up to my therapist about how I ...
19475    I need to go back to work... it takes over mos...
19477    THEORY OF MASKS\n\nSo what do i think when i v...
19478    I have no one. I've lost pretty much everythin...
19479    Long story short, my life is hard enough. A ro...
Name: Body, Length: 17285, dtype: object

,Body
0,Uk cahms service shit sure schizophrenia fucki...
1,anybody luck holistic treatment gut health met...
2,leave bed 4:20pm work 5:00pm feel like walk dr...
3,want help people care feel right like wrong bo...
4,obviously want unhappy want depressed miserabl...
...,...
19475,need work day hate work hate wake early hate c...
19476,remove
19477,THEORY MASKS \n\n think visualize mask Mask p...
19478,lose pretty cry sorry mom dad longer second pa...


16731


Topic #1: like, know, friend, think, people, thing, try, good, talk, feel like, tell, bad, person, way, love
Topic #2: year, job, school, work, start, old, find, parent, college, high, year old, money, ago, high school, new
Topic #3: time, day, come, home, leave, need, long, cry, away, night, hour, sleep, hard, break, bed
Topic #4: use, look, lose, play, game, social, world, change, mind, watch, girl, video, grow, real, video game
Topic #5: depression, help, mental, anxiety, month, health, week, mental health, issue, struggle, self, thought, diagnose, therapy, therapist
Topic #6: life, want, live, anymore, die, end, hate, shit, point, fucking, fuck, kill, stop, wish


In [31]:
topic_df = pd.DataFrame(
    model.transform(tfidf), 
    columns=["topic_{}".format(i+1) for i in range(6)]
).astype(float)

df = pd.concat([df, topic_df], axis=1)

NameError: name 'df' is not defined

In [159]:
topic_df

,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...
17546,0.0,0.0,0.0,0.0,0.0,0.0
17547,1.0,0.0,0.0,0.0,0.0,1.0
17548,0.0,1.0,1.0,1.0,1.0,0.0
17549,0.0,0.0,0.0,0.0,0.0,0.0
